In [1]:
import tensorflow as tf
import datetime
import os
import pandas as pd
import numpy as np


def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    import gzip
    labels_path = os.path.join(path,'%s-labels-idx1-ubyte.gz'% kind)
    images_path = os.path.join(path,'%s-images-idx3-ubyte.gz'% kind)
    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,offset=8)
    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 784)

    return images, labels

#load and preprocess data

dataset = 'emnist' # or 'emnist'
output_layers = 10 # default, will be overridden


if dataset == 'mnist':
    output_layers = 10
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
elif dataset == 'emnist':
    mapp = pd.read_csv("emnist/emnist-letters-mapping.txt", delimiter = ' ', \
                   index_col=0, header=None, squeeze=True)

    
    emnist_distinct_upper = ['A', 'B', 'D', 'E', 'F', 'G', 'H', 'N', 'Q', 'R', 'T'] # Characters with distinct uppercase
    emnist_nondistinct_upper = ['c', 'i', 'j', 'k', 'l', 'm', 'o', 'p', 's', 'u', 'v', 'w', 'x', 'y', 'z'] # Characters with non-distinct uppercase
    emnist_nondistinct_upper_partial = ['c', 'i', 'k', 'l', 'm', 'o', 'p', 's', 'v', 'x', 'z'] # A selection of 11 characters to use as our model can not be too big

                             # always uppercase  # Choose one of ('emnist_distinct_upper', 'emnist_nondistinct_upper_partial')
    emnist_characters = [ord(i.upper()) for i in emnist_distinct_upper]

                                        # COLUMN 1 is uppercase
    emnist_characters_ascii = list(mapp[mapp[1].isin(emnist_characters)].index)

    output_layers = len(emnist_characters_ascii)
    x_train, y_train = load_mnist('emnist', kind='emnist-letters-train') # X stands for features, y for targets
    x_test, y_test = load_mnist('emnist', kind='emnist-letters-test')
    
    train_mask = np.isin(y_train, emnist_characters_ascii)
    x_train = x_train[train_mask]
    y_train = y_train[train_mask] - 1
    test_mask = np.isin(y_test, emnist_characters_ascii)
    x_test = x_test[test_mask]
    y_test = y_test[test_mask] - 1


def rotate(image):
    image = image.reshape(28, 28, 1)
    image = np.fliplr(image)
    image = np.rot90(image)
    return image


x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = np.asarray(x_train)
x_train = np.apply_along_axis(rotate, 1, x_train)
print ("x_train:",x_train.shape)

x_test = np.asarray(x_test)
x_test = np.apply_along_axis(rotate, 1, x_test)
print ("x_test:",x_test.shape)

x_train: (52800, 28, 28, 1)
x_test: (8800, 28, 28, 1)


In [2]:
y_train_labels, y_train_values = np.unique(y_train, return_inverse=True)
y_test_labels, y_test_values = np.unique(y_test, return_inverse=True)

In [21]:
LABELS = [chr(mapp.iloc[i][1]) for i in y_train_labels]

print(LABELS)

['A', 'B', 'D', 'E', 'F', 'G', 'H', 'N', 'Q', 'R', 'T']


In [16]:
y_train_labels

array([ 0,  1,  3,  4,  5,  6,  7, 13, 16, 17, 19], dtype=uint8)

In [3]:
y_train_labels[y_train_values] ## Reconstruct

NameError: name 'y_labels' is not defined

In [6]:
# x_train = x_train.reshape(x_train.shape[0],28,28,1)
# x_test = x_test.reshape(x_test.shape[0],28,28,1)

#create model

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, kernel_size=(3,3), strides=(1,1),
                         activation='relu',input_shape=(28,28,1)),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(output_layers),
  tf.keras.layers.Softmax()
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1352)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 11)                14883     
_________________________________________________________________
softmax_1 (Softmax)          (None, 11)                0         
Total params: 14,963
Trainable params: 14,963
Non-trainable params: 0
_________________________________________________________________


In [8]:
#add datagen to make the model more accepting for the 'mouse' written digits
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False, # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.0, # Randomly zoom image 
        width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

#Train the model using the augmentation datagenerator
history = model.fit(
      datagen.flow(x_train,y_train_values, batch_size=64),
      validation_data=(x_test, y_test_values),
      steps_per_epoch=100,
      epochs=150,
      verbose=2)


#Save the resulted model
model.save(dataset + "-model_augm.h5")

Epoch 1/150
100/100 - 1s - loss: 2.1680 - accuracy: 0.4627 - val_loss: 1.9492 - val_accuracy: 0.6414
Epoch 2/150
100/100 - 1s - loss: 1.9007 - accuracy: 0.6727 - val_loss: 1.8799 - val_accuracy: 0.6845
Epoch 3/150
100/100 - 1s - loss: 1.8751 - accuracy: 0.6867 - val_loss: 1.8615 - val_accuracy: 0.6963
Epoch 4/150
100/100 - 1s - loss: 1.8445 - accuracy: 0.7155 - val_loss: 1.8444 - val_accuracy: 0.7098
Epoch 5/150
100/100 - 1s - loss: 1.8399 - accuracy: 0.7155 - val_loss: 1.8316 - val_accuracy: 0.7210
Epoch 6/150
100/100 - 1s - loss: 1.8231 - accuracy: 0.7334 - val_loss: 1.8247 - val_accuracy: 0.7287
Epoch 7/150
100/100 - 1s - loss: 1.8202 - accuracy: 0.7302 - val_loss: 1.8179 - val_accuracy: 0.7352
Epoch 8/150
100/100 - 1s - loss: 1.8156 - accuracy: 0.7361 - val_loss: 1.8061 - val_accuracy: 0.7464
Epoch 9/150
100/100 - 1s - loss: 1.8073 - accuracy: 0.7442 - val_loss: 1.8040 - val_accuracy: 0.7491
Epoch 10/150
100/100 - 1s - loss: 1.8033 - accuracy: 0.7477 - val_loss: 1.7989 - val_accura

In [23]:
#-------------------------------------------------------------------
#or quantize aware training to directly convert it to a tflite model
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# 'quantize_model' requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

train_images_subset = x_train
train_labels_subset = y_train_values

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)

converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()
open(dataset + "-quantized_model.tflite", "wb").write(quantized_tflite_model)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_1 (QuantizeLa (None, 28, 28, 1)         3         
_________________________________________________________________
quant_conv2d_1 (QuantizeWrap (None, 26, 26, 8)         99        
_________________________________________________________________
quant_max_pooling2d_1 (Quant (None, 13, 13, 8)         1         
_________________________________________________________________
quant_flatten_1 (QuantizeWra (None, 1352)              1         
_________________________________________________________________
quant_dense_1 (QuantizeWrapp (None, 11)                14888     
_________________________________________________________________
quant_softmax_1 (QuantizeWra (None, 11)                1         
Total params: 14,993
Trainable params: 14,963
Non-trainable params: 30
_________________________________________________

INFO:tensorflow:Assets written to: C:\Users\natha\AppData\Local\Temp\tmp8wb7itat\assets


INFO:tensorflow:Assets written to: C:\Users\natha\AppData\Local\Temp\tmp8wb7itat\assets


18376